### 載入MNIST手寫數字數據集

In [41]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [42]:
import tensorflow as tf

### 定義卷積神經網路函式

In [43]:
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides = [1,1,1,1],padding = 'SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')

def weight_variable(shape,name):
    return tf.Variable(tf.truncated_normal(shape,stddev = 0.1),name = name)

def bias_variable(shape,name):
    return tf.Variable(tf.constant(0.1,shape = shape),name = name)

### 定義模型變數

In [44]:
X = tf.placeholder(tf.float32,[None,784])
y_true = tf.placeholder(tf.float32,[None,10])

X_image = tf.reshape(X,[-1,28,28,1])

#### 定義第一層卷積

In [45]:
w1 = weight_variable([5,5,1,32],'w1')
b1 = bias_variable([32],'b1')
conv1 = conv2d(X_image,w1)+b1
conv1_with_act = tf.nn.relu(conv1)
output1 = maxpool2d(conv1_with_act)

#### 定義第二層卷積

In [46]:
w2 = weight_variable([5,5,32,64],'w2')
b2 = bias_variable([64],'b2')
conv2 = conv2d(output1,w2)+b2
conv2_with_act = tf.nn.relu(conv2)
output2 = maxpool2d(conv2_with_act)

#### 定義撫平層

In [47]:
flatten_output = tf.reshape(output2,[-1,7*7*64])

#### 定義全連接層

In [48]:
fc_w1 = weight_variable([7*7*64,1024],'fc_w1')
fc_b1 = bias_variable([1024],'fc_b1')
fc_o1 = tf.nn.relu(tf.matmul(flatten_output,fc_w1)+fc_b1)

### 定義Dropout

In [49]:
keep_prob = tf.placeholder(tf.float32)
fc_o1_with_dropout = tf.nn.dropout(fc_o1,keep_prob)

#### 定義輸出層

In [50]:
fc_w2 = weight_variable([1024,10],'fc_w2')
fc_b2 = bias_variable([10],'fc_b2')
fc_o2 = tf.matmul(fc_o1_with_dropout,fc_w2)+fc_b2

### 定義損失函數與優化器

In [51]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = fc_o2,labels = y_true))
opt = tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_prediction = tf.equal(tf.argmax(fc_o2,1),tf.arg_max(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

### 實例化與執行

In [53]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        batch_x,batch_y = mnist.train.next_batch(100)
        
        if step % 100 == 0:
            print("Step:{},Accuracy:{}".format(step,accuracy.eval(feed_dict = {X:batch_x,y_true:batch_y,keep_prob:1.0})))
        
        sess.run(opt,feed_dict = {X:batch_x,y_true:batch_y,keep_prob:0.5})
    
    print("Test Accuracy:{}".format(sess.run(accuracy,feed_dict = {X:mnist.test.images,y_true:mnist.test.labels,keep_prob:1.0})))

Step:0,Accuracy:0.10000000149011612
Step:100,Accuracy:0.9399999976158142
Step:200,Accuracy:0.9100000262260437
Step:300,Accuracy:0.9399999976158142
Step:400,Accuracy:0.9300000071525574
Step:500,Accuracy:0.9399999976158142
Step:600,Accuracy:0.9599999785423279
Step:700,Accuracy:0.949999988079071
Step:800,Accuracy:0.9700000286102295
Step:900,Accuracy:0.949999988079071
Test Accuracy:0.9685999751091003
